In [59]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_c630b40987034e00a2c13458fdb2cc88 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='5-7nMriVnLgZfwshJWjfILlgaJeG_C56eDPoe8AJTFCR',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_c630b40987034e00a2c13458fdb2cc88.get_object(Bucket='5512-donotdelete-pr-qtcqvweu0xfryc',Key='reddit_train.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

train = pd.read_csv(body)
train.head()
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

sns.set()

## 1 - Open training data

In [60]:
#train_data = "reddit_train.csv"
#train = pd.read_csv(train_data)
train.head()

,id,comments,subreddits
0,0,"Honestly, Buffalo is the correct answer. I rem...",hockey
1,1,Ah yes way could have been :( remember when he...,nba
2,2,https://youtu.be/6xxbBR8iSZ0?t=40m49s\n\nIf yo...,leagueoflegends
3,3,He wouldn't have been a bad signing if we woul...,soccer
4,4,Easy. You use the piss and dry technique. Let ...,funny


## 2 - Basic Feature Extraction
 

### Number of Words

In [61]:
train['word_count'] = train['comments'].apply(lambda x: len(str(x).split(" ")))
train.head()

,id,comments,subreddits,word_count
0,0,"Honestly, Buffalo is the correct answer. I rem...",hockey,58
1,1,Ah yes way could have been :( remember when he...,nba,29
2,2,https://youtu.be/6xxbBR8iSZ0?t=40m49s\n\nIf yo...,leagueoflegends,18
3,3,He wouldn't have been a bad signing if we woul...,soccer,24
4,4,Easy. You use the piss and dry technique. Let ...,funny,46


### 2.1 Number of Characters

In [62]:
train['char_count'] = train['comments'].str.len() ## this also includes spaces
train.head()

,id,comments,subreddits,word_count,char_count
0,0,"Honestly, Buffalo is the correct answer. I rem...",hockey,58,357
1,1,Ah yes way could have been :( remember when he...,nba,29,145
2,2,https://youtu.be/6xxbBR8iSZ0?t=40m49s\n\nIf yo...,leagueoflegends,18,145
3,3,He wouldn't have been a bad signing if we woul...,soccer,24,123
4,4,Easy. You use the piss and dry technique. Let ...,funny,46,212


### 2.2 Removing the stopwords

In [63]:
# do this if its the first time using nltk stopwords
import nltk
nltk.download('stopwords') 

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/dsxuser/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [64]:
from nltk.corpus import stopwords

stop = stopwords.words('english')

train['stopwords'] = train['comments'].apply(lambda x: len([x for x in x.split() if x in stop]))
train.head()

,id,comments,subreddits,word_count,char_count,stopwords
0,0,"Honestly, Buffalo is the correct answer. I rem...",hockey,58,357,20
1,1,Ah yes way could have been :( remember when he...,nba,29,145,12
2,2,https://youtu.be/6xxbBR8iSZ0?t=40m49s\n\nIf yo...,leagueoflegends,18,145,9
3,3,He wouldn't have been a bad signing if we woul...,soccer,24,123,12
4,4,Easy. You use the piss and dry technique. Let ...,funny,46,212,17


### 2.3 Does the comment contain a url ?

In [65]:
train['links'] = train['comments'].apply(lambda x: True if "http" in x else False)
train.head()

,id,comments,subreddits,word_count,char_count,stopwords,links
0,0,"Honestly, Buffalo is the correct answer. I rem...",hockey,58,357,20,False
1,1,Ah yes way could have been :( remember when he...,nba,29,145,12,False
2,2,https://youtu.be/6xxbBR8iSZ0?t=40m49s\n\nIf yo...,leagueoflegends,18,145,9,True
3,3,He wouldn't have been a bad signing if we woul...,soccer,24,123,12,False
4,4,Easy. You use the piss and dry technique. Let ...,funny,46,212,17,False


### 2.4 Number of Numerics

In [66]:
train['numerics'] = train['comments'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))
train.head()

,id,comments,subreddits,word_count,char_count,stopwords,links,numerics
0,0,"Honestly, Buffalo is the correct answer. I rem...",hockey,58,357,20,False,1
1,1,Ah yes way could have been :( remember when he...,nba,29,145,12,False,0
2,2,https://youtu.be/6xxbBR8iSZ0?t=40m49s\n\nIf yo...,leagueoflegends,18,145,9,True,0
3,3,He wouldn't have been a bad signing if we woul...,soccer,24,123,12,False,0
4,4,Easy. You use the piss and dry technique. Let ...,funny,46,212,17,False,0


### 2.5 Number of Uppercase words

In [67]:
train['upper'] = train['comments'].apply(lambda x: len([x for x in x.split() if x.isupper()]))
train.head()

,id,comments,subreddits,word_count,char_count,stopwords,links,numerics,upper
0,0,"Honestly, Buffalo is the correct answer. I rem...",hockey,58,357,20,False,1,2
1,1,Ah yes way could have been :( remember when he...,nba,29,145,12,False,0,1
2,2,https://youtu.be/6xxbBR8iSZ0?t=40m49s\n\nIf yo...,leagueoflegends,18,145,9,True,0,0
3,3,He wouldn't have been a bad signing if we woul...,soccer,24,123,12,False,0,1
4,4,Easy. You use the piss and dry technique. Let ...,funny,46,212,17,False,0,0


## 3 - Basic Pre-processing

In [68]:
# Keep a copy of the unprocessed training dataframe just in case
unprocessed_train = train.copy()

### 3.1 - Lowercase

In [69]:
train['comments'] = train['comments'].apply(lambda x: " ".join(x.lower() for x in x.split()))
train.head()

,id,comments,subreddits,word_count,char_count,stopwords,links,numerics,upper
0,0,"honestly, buffalo is the correct answer. i rem...",hockey,58,357,20,False,1,2
1,1,ah yes way could have been :( remember when he...,nba,29,145,12,False,0,1
2,2,https://youtu.be/6xxbbr8isz0?t=40m49s if you d...,leagueoflegends,18,145,9,True,0,0
3,3,he wouldn't have been a bad signing if we woul...,soccer,24,123,12,False,0,1
4,4,easy. you use the piss and dry technique. let ...,funny,46,212,17,False,0,0


### 3.2 Removing Punctuation

In [70]:
train['comments'] = train['comments'].str.replace('[^\w\s]','')
train.head()

,id,comments,subreddits,word_count,char_count,stopwords,links,numerics,upper
0,0,honestly buffalo is the correct answer i remem...,hockey,58,357,20,False,1,2
1,1,ah yes way could have been remember when he w...,nba,29,145,12,False,0,1
2,2,httpsyoutube6xxbbr8isz0t40m49s if you didnt fi...,leagueoflegends,18,145,9,True,0,0
3,3,he wouldnt have been a bad signing if we would...,soccer,24,123,12,False,0,1
4,4,easy you use the piss and dry technique let a ...,funny,46,212,17,False,0,0


### 3.2 Removal of Stop Words

In [71]:
train['comments'] = train['comments'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
train.head()

,id,comments,subreddits,word_count,char_count,stopwords,links,numerics,upper
0,0,honestly buffalo correct answer remember peopl...,hockey,58,357,20,False,1,2
1,1,ah yes way could remember drafted thought gonn...,nba,29,145,12,False,0,1
2,2,httpsyoutube6xxbbr8isz0t40m49s didnt find alre...,leagueoflegends,18,145,9,True,0,0
3,3,wouldnt bad signing wouldnt paid 18m euros rig...,soccer,24,123,12,False,0,1
4,4,easy use piss dry technique let drops let dry ...,funny,46,212,17,False,0,0


### 3.3 Spelling correction - WARNING: takes a very long time!

In [72]:
!pip install -U textblob
############not running this code because it takes so much time

Requirement already up-to-date: textblob in /opt/conda/envs/Python36/lib/python3.6/site-packages (0.15.3)


from textblob import TextBlob
train['comments'].apply(lambda x: str(TextBlob(x).correct()))
train.head()

## 3.4 tokenizing and lemmatization 

In [73]:
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('punkt')


In [74]:
train['comments'] = train['comments'].apply(lambda x: word_tokenize(x))


In [75]:
ps = PorterStemmer()

train['comments'] = train['comments'].apply(lambda x:" ".join(ps.stem(word)for word in x))


In [76]:
train.head()

,id,comments,subreddits,word_count,char_count,stopwords,links,numerics,upper
0,0,honestli buffalo correct answer rememb peopl s...,hockey,58,357,20,False,1,2
1,1,ah ye way could rememb draft thought gon na gr...,nba,29,145,12,False,0,1
2,2,httpsyoutube6xxbbr8isz0t40m49 didnt find alrea...,leagueoflegends,18,145,9,True,0,0
3,3,wouldnt bad sign wouldnt paid 18m euro right p...,soccer,24,123,12,False,0,1
4,4,easi use piss dri techniqu let drop let dri ri...,funny,46,212,17,False,0,0


In [77]:
targets=train["subreddits"]
traindata=train["comments"]
len(targets)
len(traindata)

70000

In [78]:
#######test data
body = client_c630b40987034e00a2c13458fdb2cc88.get_object(Bucket='5512-donotdelete-pr-qtcqvweu0xfryc',Key='reddit_test.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

reddit_test = pd.read_csv(body)
reddit_test.head()


,id,comments
0,0,Trout and Bryant have both led the league in s...
1,1,&gt; Just like Estonians have good reasons to ...
2,2,Will Sol_Primeval sotp being oblivious?\n\nfin...
3,3,Moving Ostwald borders back to the pre 1967 bo...
4,4,"You have to take it out of the bag, Morty!"


In [79]:
X_train, X_test, y_train, y_test = train_test_split(traindata, targets, test_size=0.15)


## Building a pipeline
#### vectorizer => transformer => classifier
##### multinomial naive bayes


In [80]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np


from sklearn.pipeline import Pipeline
text_clf = Pipeline([
     ('vect', CountVectorizer()),
     ('tfidf', TfidfTransformer()),
     ('clf', MultinomialNB()),
 ])

text_clf.fit(X_train, y_train) 

np.mean(y_test==text_clf.predict(X_test))

0.5578095238095238

### Building a pipeline
##### Support vector machine

In [81]:
from sklearn.linear_model import SGDClassifier
>>> text_clf = Pipeline([
 ('vect', CountVectorizer()),
('tfidf', TfidfTransformer()),
 ('clf', SGDClassifier(loss='hinge', penalty='l2',
                           alpha=1e-3, random_state=42,
                           max_iter=5, tol=None)),
])
text_clf.fit(X_train, y_train) 
np.mean(y_test==text_clf.predict(X_test))

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


0.5381904761904762